In [1]:
from model import PointGen
import torch
# torch.manual_seed(20)

<h3>Dimensionality check</h3>
<p>
Model should generate a (1, 1024, 3) and contain about 40m parameters. It took around 200ms to process one image in my laptop.
</p>

In [2]:
model = PointGen()
# print(model)
print('Number of parameters: ', sum(p.numel() for p in model.parameters()))
x = torch.rand(1, 3, 224, 224)
model.eval()
model(x)  # cold start
%time ys = model(x)
print([y.size() for y in ys])

Number of parameters:  45757651
CPU times: user 893 ms, sys: 476 ms, total: 1.37 s
Wall time: 201 ms
[torch.Size([1, 1024, 3]), torch.Size([1, 1024, 3])]


<h3>Data loader checks</h3>

In [3]:
from data import ModelNet40
from data.visualize import visualize_points
import open3d as o3d

data = ModelNet40(points_to_sample=1024)

In [ ]:
d = data[0]
visualize_points(d[-1])

<h3>Overfitting Experiments</h3>

<p>Here, we sample a small set of data and try to overfit.</p>

In [2]:
import torch
from data import ModelNet40
data = ModelNet40(points_to_sample=1024)

In [13]:
print(len(data))
data_size=8
idx = list(range(data_size))# torch.randperm(len(data))[:data_size].tolist()
print(idx)

118116
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [14]:
small_data = [data[i] for i in idx]
coarse_points = torch.stack([d[1] for d in small_data])
detailed_points = torch.stack([d[2] for d in small_data])
images = torch.stack([d[0] for d in small_data])

<h4>Overfitting the coarse predictor</h4>
<p>Here, we try to overfit the coarse prediction layer</p>

In [15]:
print(images.shape, coarse_points.shape)
'''images = (images - images.mean(dim=[0, 2, 3], keepdim=True)) \
    / images.std(dim=[0, 2, 3], keepdim=True).add(1e-5)
print(images.mean(dim=[0,2,3]))
print(images.std(dim=[0,2,3]))'''


torch.Size([16, 3, 224, 224]) torch.Size([16, 256, 3])


'images = (images - images.mean(dim=[0, 2, 3], keepdim=True))     / images.std(dim=[0, 2, 3], keepdim=True).add(1e-5)\nprint(images.mean(dim=[0,2,3]))\nprint(images.std(dim=[0,2,3]))'

In [16]:
from model.encoder import Encoder
from model.decoder_fold import CoarseDecoder
from model import PointGen
from model.loss import chamfer_distance, std_loss
from torch.optim import Adam, SGD
import torch.nn as nn

In [17]:
net = PointGen()
#print(net)
opt = Adam(net.parameters(), lr=1e-2)
batch_size = 8
epochs = 50
for e in range(epochs):
    idx = torch.randperm(data_size).tolist()
    for i in range(0, data_size, batch_size):
        yc = coarse_points[i:i+batch_size]
        yd = detailed_points[i:i+batch_size]
        x = images[i:i+batch_size]
        opt.zero_grad()
        yc_pred, yd_pred = net(x)
        # lossc = chamfer_distance(yc_pred, yc)
        lossd = chamfer_distance(yd_pred, yd)
        lossd.backward()
        opt.step()
        # print('Loss: ', loss)
    with torch.no_grad():
        net.eval()
        preds = net(images)
        print('Epoch', e)
        # print('Loss Coarse', chamfer_distance(preds[0], coarse_points).item())
        print('Loss Detailed', chamfer_distance(preds[1], detailed_points).item())
    net.train()

Epoch 0
Loss Detailed 0.07852254807949066
Epoch 1
Loss Detailed 0.06780120730400085
Epoch 2
Loss Detailed 0.05308149755001068


KeyboardInterrupt: 

In [8]:
net.eval()
for i in range(data_size):
    img = images[i].unsqueeze(0)
    pc = detailed_points[i].unsqueeze(0)
    pred = net(img)[1]
    print('{} Distance: {}'.format(i, chamfer_distance(pred, pc)))

0 Distance: 0.0013017186429351568
1 Distance: 0.0014305756194517016
2 Distance: 0.001511970884166658
3 Distance: 0.0018123614136129618
4 Distance: 0.0016230328474193811
5 Distance: 0.0018113504629582167
6 Distance: 0.0017678458243608475
7 Distance: 0.001395365223288536


In [ ]:
torch.save(net.state_dict(), 'overfit2')

In [11]:
from data.visualize import visualize_points
# visualize_points(data[0][1])
net.eval()
with torch.no_grad():
    pred = net(images[1].unsqueeze(0))[1]
pred = pred.detach().contiguous().squeeze()
print(pred.shape)
visualize_points(pred)
visualize_points(detailed_points[1])

torch.Size([1024, 3])


In [ ]:
pcd_load = o3d.io.read_point_cloud("view0.ply")
pcd_load2 = o3d.io.read_point_cloud("view0_pred.ply")
o3d.visualization.draw_geometries([pcd_load])

In [ ]:

o3d.visualization.draw_geometries([pcd_load2])